Generates ranges of CNN models, saves as *h5* files. <br />
Software License Agreement (MIT License) <br />
Copyright (c) 2020, Amirhossein Pakdaman.

In [ ]:
import tensorflow as tf
import keras
import keras.layers as kl
from keras import backend as K
import numpy as np
import os
from matplotlib import pyplot as plt

In [ ]:
def conv2d(blocks, sizes, inputs, outputs):
    layers = [kl.Conv2D(sizes, (3, 3), input_shape = (inputs, inputs, 3),strides=(1, 1), padding='same', activation = 'relu'),
              kl.MaxPooling2D(pool_size = (2, 2))]

    for i in range(blocks-1):
        layers.append( kl.Conv2D(sizes, (3, 3),strides=(1, 1), padding='same', activation = 'relu') )
        layers.append( kl.MaxPooling2D(pool_size = (2, 2)) ) 

    layers.append( kl.Flatten() )
    layers.append( kl.Dense(units = outputs, activation = 'softmax') )
      
    name = 'Conv2D_'+ str(blocks) +'_'+ str(sizes) +'_'+ str(inputs) +'_'+ str(outputs) +'_'
    
    return layers, name

In [ ]:
def depthwiseconv2d(blocks, inputs, outputs):
    layers = [kl.DepthwiseConv2D( (3, 3), input_shape = (inputs, inputs, 3),strides=(1, 1), padding='same', activation = 'relu'),
              kl.MaxPooling2D(pool_size = (2, 2))]

    for i in range(blocks-1):
        layers.append( kl.DepthwiseConv2D( (3, 3),strides=(1, 1), padding='same', activation = 'relu') )
        layers.append( kl.MaxPooling2D(pool_size = (2, 2)) ) 

    layers.append( kl.Flatten() )
    layers.append( kl.Dense(units = outputs, activation = 'softmax') )
    
    name = 'DepthwiseConv2D_'+ str(blocks) +'_'+ 'x' +'_'+ str(inputs) +'_'+ str(outputs) +'_'    
    
    return layers, name

In [ ]:
def fullyconnected(blocks, sizes, inputs, outputs):
    inp = kl.Input(shape=(inputs,inputs))
    a1 = kl.Flatten()(inp)
    for i in range(blocks-1):
        a1 = kl.Dense(units = sizes, activation = 'relu')(a1)
    otp = kl.Dense(units = outputs, activation = 'softmax')(a1)   
    
    name = 'FullyConnected_'+ str(blocks) +'_'+ str(sizes) +'_'+ str(inputs) +'_'+ str(outputs) +'_'
    
    return inp, otp, name

In [ ]:
modelMaxSize = 3500000
typeModel = 'Conv2D'

blocksS = 2
blocksE = 6
blocksI = 1

filtersS = 34
filtersE = 42
filtersI = 4

imagesS = 16
imagesE = 224
imagesI = 52

outputsS = 2
outputsE = 10
outputsI = 4

cnt = 0
for outputs in range(outputsS, outputsE+1, outputsI):
    for images in range(imagesS, imagesE+1, imagesI):
        for filters in range(filtersS, filtersE+1, filtersI):
            for blocks in range(blocksS, blocksE+1, blocksI):
                if (images / 2**blocks) >= 4:
                        layers = []
                        if typeModel == 'Conv2D':
                            layers, name = conv2d(blocks, filters, images, outputs)
                            net = keras.Sequential(layers)
                        elif typeModel == 'DepthwiseConv2D':
                            layers, name = depthwiseconv2d(blocks, images, outputs)
                            net = keras.Sequential(layers)
                        net.compile(loss='categorical_crossentropy', optimizer='sgd', metrics = ['accuracy'])
                        nameFile = 'models/' + name + str(net.count_params()) + '.h5'
                        net.save(nameFile)
                        fileStat = os.stat(nameFile)
                        print(nameFile, '-', fileStat.st_size/1000, '- saved')
                        cnt += 1
                        if fileStat.st_size > modelMaxSize:
                            os.remove(nameFile)
                            print(nameFile, '-', fileStat.st_size/1000, '- removed')
                            cnt -= 1
print(cnt)

In [ ]:
modelMaxSize = 3500000
typeModel = 'DepthwiseConv2D' 

blocksS = 1
blocksE = 5
blocksI = 1

imagesS = 64
imagesE = 224
imagesI = 32

outputsS = 2
outputsE = 82
outputsI = 16

cnt = 0
for outputs in range(outputsS, outputsE+1, outputsI):
    for images in range(imagesS, imagesE+1, imagesI):
        for blocks in range(blocksS, blocksE+1, blocksI):
            if (images / 2**blocks) >= 4:
                    layers = []
                    if typeModel == 'Conv2D':
                        layers, name = conv2d(blocks, filters, images, outputs)
                        net = keras.Sequential(layers)
                    elif typeModel == 'DepthwiseConv2D':
                        layers, name = depthwiseconv2d(blocks, images, outputs)
                        net = keras.Sequential(layers)
                    net.compile(loss='categorical_crossentropy', optimizer='sgd', metrics = ['accuracy'])
                    nameFile = 'models/' + name + str(net.count_params()) + '.h5'
                    net.save(nameFile)
                    fileStat = os.stat(nameFile)
                    print(nameFile, '-', fileStat.st_size/1000, '- saved')
                    cnt += 1
                    if fileStat.st_size > modelMaxSize:
                        os.remove(nameFile)
                        print(nameFile, '-', fileStat.st_size/1000, '- removed')
                        cnt -= 1
print(cnt)

In [ ]:
blk = []
flt = []
img = []
otp = []
par = []
siz = []

for filename in os.listdir('models'):
    data = filename.replace('.h5','').split('_')
    blk.append(data[1])
    flt.append(data[2])
    img.append(data[3])
    otp.append(data[4])
    par.append(data[5])
    fileStat = os.stat('models/' + filename)
    siz.append(fileStat.st_size/1000)

In [ ]:
siz.sort()
plt.plot(siz)
plt.show()